In [1]:
import pandas as pd 
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('Provider Rating Data.csv')

In [3]:
df.head()

,NPI,PAC ID,Last Name,First Name,Gender,Line 1 Street Address,City,State,Zip Code,Phone Number,Overall Rating
0,1003004573,6103963236,OSETSKY,JOANNE,F,7320 216TH ST SW,EDMONDS,WA,980268006,2.063203e+09,34.500000
1,1003013236,42305245,OSTASZEWSKI,MARY,F,4720 NELSON BROGDON BLVD,SUGAR HILL,GA,305183480,7.709452e+09,94.166667
2,1003015140,7416039250,BAUER,JENNY,F,5435 FELTL RD,MINNETONKA,MN,553437983,9.528360e+09,30.000000
3,1003021700,4183770530,GUERRA,SUSAN,F,2106 NEW RD,LINWOOD,NJ,82211050,6.099269e+09,40.000000
4,1003041104,941332738,O'CALLAGHAN,BARBARA,F,1000 MEDICAL CTR BLVD,LAWRENCEVILLE,GA,300467694,6.784426e+09,100.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10829 entries, 0 to 10828
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   NPI                    10829 non-null  int64  
 1   PAC ID                 10829 non-null  int64  
 2   Last Name              10829 non-null  object 
 3   First Name             10829 non-null  object 
 4   Gender                 10829 non-null  object 
 5   Line 1 Street Address  10829 non-null  object 
 6   City                   10829 non-null  object 
 7   State                  10829 non-null  object 
 8   Zip Code               10829 non-null  int64  
 9   Phone Number           10134 non-null  float64
 10  Overall Rating         10829 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 930.7+ KB


In [5]:
df = df.rename(columns={'Zip Code': 'Zip_Code', 'Overall Rating': 'Overall_Rating','Last Name':'Last_Name','Phone Number':'Phone_Number'})

In [6]:
df.head()

,NPI,PAC ID,Last_Name,First Name,Gender,Line 1 Street Address,City,State,Zip_Code,Phone_Number,Overall_Rating
0,1003004573,6103963236,OSETSKY,JOANNE,F,7320 216TH ST SW,EDMONDS,WA,980268006,2.063203e+09,34.500000
1,1003013236,42305245,OSTASZEWSKI,MARY,F,4720 NELSON BROGDON BLVD,SUGAR HILL,GA,305183480,7.709452e+09,94.166667
2,1003015140,7416039250,BAUER,JENNY,F,5435 FELTL RD,MINNETONKA,MN,553437983,9.528360e+09,30.000000
3,1003021700,4183770530,GUERRA,SUSAN,F,2106 NEW RD,LINWOOD,NJ,82211050,6.099269e+09,40.000000
4,1003041104,941332738,O'CALLAGHAN,BARBARA,F,1000 MEDICAL CTR BLVD,LAWRENCEVILLE,GA,300467694,6.784426e+09,100.000000


In [7]:
df["Zip_Code"] = df["Zip_Code"].astype(str)

In [8]:
# Step 2: Grouping the data by zip code and rating
grouped = df.groupby(["Zip_Code", "Overall_Rating"])["Last_Name", "Phone_Number","Gender"] \
          .agg({"Last_Name": list, "Phone_Number": list,'Gender':list}).reset_index()

grouped["avg_rating"] = grouped.groupby("Zip_Code")["Overall_Rating"].transform("mean")

C:\Users\shash\AppData\Local\Temp\ipykernel_33744\1630961007.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped = df.groupby(["Zip_Code", "Overall_Rating"])["Last_Name", "Phone_Number","Gender"] \


In [9]:
# Step 3: Calculating similarity between nurses
features = grouped[["Zip_Code", "Overall_Rating", "avg_rating"]]
X = pd.get_dummies(features)
similarity_matrix = cosine_similarity(X)

In [10]:
def recommend_nurses(Zip_Code, top_n=5):
    index = grouped[grouped["Zip_Code"] == Zip_Code].index.values[0]
    similarity_scores = similarity_matrix[index]
    similar_nurses_index = similarity_scores.argsort()[::-1][1:top_n+1]
    similar_nurses = grouped.loc[similar_nurses_index, ["Last_Name", "Overall_Rating","Phone_Number","Gender"]]
    similar_nurses_sorted = similar_nurses.sort_values(by="Overall_Rating", ascending=False)
    return similar_nurses_sorted.to_dict(orient="records")


In [12]:
# Call the recommend_doctors function
similar_nurses = recommend_nurses("305183480",top_n=5)
print(similar_nurses)


[{'Last_Name': ['CAIN'], 'Overall_Rating': 94.0, 'Phone_Number': [8159712000.0], 'Gender': ['F']}, {'Last_Name': ['MCENEANEY'], 'Overall_Rating': 93.3333333333333, 'Phone_Number': [2123054098.0], 'Gender': ['F']}, {'Last_Name': ['BOSTON'], 'Overall_Rating': 92.4, 'Phone_Number': [5023279100.0], 'Gender': ['F']}, {'Last_Name': ['PEYNADO'], 'Overall_Rating': 92.0, 'Phone_Number': [9563896565.0], 'Gender': ['F']}, {'Last_Name': ['TRACY'], 'Overall_Rating': 92.0, 'Phone_Number': [8455614400.0], 'Gender': ['F']}]
